# Jupyter Version of the Ironpython code from Dynamo

## Importing packages

In [ ]:
import math
import random
from itertools import repeat
import statistics
import csv
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Starting from similar inputs

In [ ]:
# list of all possible rectangles

ml = 6 # module length
mw = 4 # module width
mh = 3 # module height

bl = 65 # building length
bw = 55 # building width
bh = 100 #building height

modx = math.floor(bl / ml) # number of modules on X axis
mody = math.floor(bw / mw) # number of modules on Y axis
modz = math.floor(bh / mh) # number of modules on Z axis

recs = [[[1 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]

# exceptions are not considered

# maximal area
MGF = 34500 / 0.90
# weights for the gravitational model
weights = [- 0.2, 0.4, 0.45, 0.15]
# random seed
random.seed(1537)

## Finding the center Points

In [ ]:
def get_center(rectangles, ml, mw, mh):
    coords = []
    for level in range(len(rectangles)):
        Z = level * mh
        coordslevel = []
        for row in range(len(rectangles[level])):
            Y = row * mw + mw / 2 # center point of the rectangle
            coordsrow = []
            for rec in range(len(rectangles[level][row])):
                X = rec * ml + ml / 2 #center point of the rectangle
                # recs is the list of possible modules, therefore all the values are 1
                # recsel is the selection of modules according to multiple rules, it has 1 and 0 
                if rectangles[level][row][rec] == 1:
                    coordsrow.append([X, Y, Z])
            coordslevel.append(coordsrow)
        coords.append(coordslevel)
    return(coords)

In [ ]:
coords = get_center(recs, 6, 4, 3)

In [ ]:
corners = [coords[0][0][0], coords[0][0][-1], coords[0][-1][0], coords[0][-1][-1]]
corners

## Distance matrix

here the program calculates the distance to each module center to the corners

In [ ]:
dm = [[[[[] for x in repeat(None, 4)] for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
for level in range(len(coords)):
    for row in range(len(coords[level])):
        for el in range(len(coords[level][row])):
            X0 = coords[level][row][el][0]
            Y0 = coords[level][row][el][1]
            for c in range(len(corners)):
                d = math.sqrt((X0 - corners[c][0]) ** 2 + (Y0 - corners[c][1]) ** 2)
                # print(d)
                dm[level][row][el][c] = d

### Function to get distance matrix

In [ ]:
def distanceMatrix(coords, corners = True, gravityCenters = 4, dimensions = 2):
    
    # this function calculates the distance of each point to the corner points, given the following structure of coords
    # coords = [[[P111[x,y,z], P112[x,y,z], ..., P11N[x,y,z]], [P121[x,y,z], P122[x,y,z], ..., P12N[x, y, z]], ... ]
    
    
    # number of repetitions on z axis
    repz = len(coords)
    # number of repetitions on y axis
    repy = len(coords[0])
    # number of repetitions on x axis
    repx = len(coords[0][0])
    # corners
    if corners == True:
        gravityPoints = [coords[0][0][0], coords[0][0][-1], coords[0][-1][0], coords[0][-1][-1]]
    else:
        gravityPoints = []
        for g in range(gravityCenters):
            selx = random.sample(list(range(repx)), 1)
            sely = random.sample(list(range(repy)), 1)
            selz = random.sample(list(range(repz)), 1)
            gravityPoints.append(coords[selz[0]][sely[0]][selx[0]])
    
    # output
    dm = [[[[[] for x in repeat(None, gravityCenters)] for x in repeat(None, repx)] for x in repeat(None, repy)] for x in repeat(None, repz)]
    
    # main loop
    for level in range(len(coords)):
        for row in range(len(coords[level])):
            for el in range(len(coords[level][row])):
                X0 = coords[level][row][el][0]
                Y0 = coords[level][row][el][1]
                Z0 = coords[level][row][el][2]
                for c in range(len(gravityPoints)):
                    if dimensions == 2:
                        d = math.sqrt((X0 - gravityPoints[c][0]) ** 2 + (Y0 - gravityPoints[c][1]) ** 2)
                    if dimensions == 3:
                        d = math.sqrt((X0 - gravityPoints[c][0]) ** 2 + (Y0 - gravityPoints[c][1]) ** 2 + (Z0 - gravityPoints[c][2]) ** 2)
                    # print(d)
                    dm[level][row][el][c] = d
    return(dm)

## Gravitation cohefficients

In [ ]:
gr = [[[[] for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
gforce = 0.1
for level in range(len(dm)):
    for row in range(len(dm[level])):
        for el in range(len(dm[level][row])):
            distances = dm[level][row][el]
            gravity = weights[0] * math.exp(-gforce * distances[0]) + weights[1] * math.exp(-gforce * distances[1]) + weights[2] * math.exp(-gforce * distances[2]) + weights[3] * math.exp(-gforce * distances[3])
            gr[level][row][el] = gravity

### Function to get gravity cohefficients

In [ ]:
def gravitycoehfficients(coords, gforce, weights, corners = True, gravityCenters = 4, dim = 2):
    
    # INPUT
    # coords
    # gforce
    # weights
    
    # OUTPUT
    # gr
    
    # number of repetitions on z axis
    repz = len(coords)
    # number of repetitions on y axis
    repy = len(coords[0])
    # number of repetitions on x axis
    repx = len(coords[0][0])
    
    gr = [[[[] for x in repeat(None, repx)] for x in repeat(None, repy)] for x in repeat(None, repz)]
    
    dm = distanceMatrix(coords, corners = corners, gravityCenters = gravityCenters, dimensions = dim)
    
    for level in range(repz):
        for row in range(repy):
            for el in range(repx):
                distances = dm[level][row][el]
                gravity = 0
                for d in range(len(distances)):
                    gravity += weights[d] * math.exp(-gforce * distances[d])
                gr[level][row][el] = gravity
    return (gr)

In [ ]:
gr = gravitycoehfficients(coords, 0.1, [-0.2, 0.4, 0.4, 0.4], 3)

In [ ]:
grflat = [item for sublist in [item for sublist in gr for item in sublist] for item in sublist]
print(statistics.mean(grflat), statistics.stdev(grflat), max(grflat), min(grflat))

In [ ]:
plt.hist(grflat, bins = 50);

## Handling Exceptions in the shape of the building

This function set a 0 in the modules matrix if a module lies in the exception space

In [ ]:
def get_exceptions(obj, x, y, z, l, w, h):
    for level in range(len(obj)):
        for row in range(len(obj[level])):
            for el in range(len(obj[level][row])):
                condx = el >= x and el < x + l
                condy = row >= y and row < y + w
                condz = level >= z and level < z + h
                if (condx and condy and condz):
                    obj[level][row][el] = 0
    return(obj)

In [ ]:
get_exceptions(recs, 4, 0, 0, 3, 10, 2)

## Cantilever Rule

A module is possible only if there is another one on the level below, exactly in the same position or just moved N step aside

In [ ]:
def cantilever(obj, steps = 1):
    length = len(obj[0][0])
    width = len(obj[0])
    for level in range(len(obj)):
        for row in range(width):
            for el in range(length):
                if level > 0:
                    sostain = obj[level - 1][row][el]
                    for s in range(1, steps + 1):
                        if (el + s in list(range(length))):
                            sostain += obj[level - 1][row][el + s]
                        if (el - s in list(range(length))):
                            sostain += obj[level - 1][row][el - s]
                        if (row + s in list(range(width))):
                            sostain += obj[level - 1][row + s][el]
                        if (row - s in list(range(width))):
                            sostain += obj[level - 1][row - s][el]
                    if sostain == 0:
                        obj[level][row][el] = 0
    return(obj)
# def cantilever(obj):
#     for level in range(len(obj)):
#         for row in range(len(obj[level])):
#             for el in range(len(obj[level][row])):
#                 if level > 0:
#                     cond1 = obj[level - 1][row][el] > 0
#                     if row == 0:
#                         cond2 = False
#                     else:
#                         cond2 = obj[level - 1][row - 1][el] > 0
#                     if row == len(obj[level]) - 1:
#                         cond3 = False
#                     else:
#                         cond3 = obj[level - 1][row + 1][el] > 0
#                     if el == 0:
#                         cond4 = False
#                     else:
#                         cond4 = obj[level - 1][row][el - 1] > 0
#                     if el == len(obj[level][row]) - 1:
#                         cond5 = False
#                     else:
#                         cond5 = obj[level - 1][row][el + 1] > 0
#                     if not (cond1 or cond2 or cond3 or cond4 or cond5):
#                         obj[level][row][el] = 0
#     return(obj)

## Compact building rule

It doesn't make sense to have an empty space in the core of the building. If there is an empty module in the middle of the structure, it will turn to a full module (from 0 to 1)

In [ ]:
def compact_building(obj):
    for level in range(len(obj)):
        for row in range(len(obj[level])):
            for el in range(len(obj[level][row])):
                # avoiding the perimeter modules
                if (el > 0 and el < len(obj[level][row]) - 1 and row > 0 and row < len(obj[level]) - 1):
                    cond1 = obj[level][row][el - 1] == 0
                    cond2 = obj[level][row][el + 1] == 0
                    cond3 = obj[level][row + 1][el] == 0
                    cond4 = obj[level][row - 1][el] == 0
                    if not (cond1 or cond2 or cond3 or cond4):
                        obj[level][row][el] = 1
    return(obj)

## Building depth rule

the idea is that room with a certain depth do not receive enough natural light and ventilation

In [ ]:
# Hard Rule: no module after x distance from the border 
# Pratically it should create a courtyard if the building results too large
def roomdepth(recs, mods):
    for level in range(len(recs)):
        for row in range(len(recs[level])):
            # if the module is not close to a Y border
            if row >= mods and row < len(recs[level]) - mods:
                for el in range(len(recs[level][row])):
                    #if a module is not close to a X border
                    if el >= mods and el < len(recs[level][row]) - mods:
                        # hard rule
                        recs[level][row][el] = 0
    return(recs)

In [ ]:
# Soft Rule: no module after a certain distance to the border or to an empty slot (0)
def softroomdepth(recs, mods):
    for level in range(len(recs)):
        for row in range(len(recs[level])):
            # if the module is not close to a Y border
            if row >= mods and row < len(recs[level]) - mods:
                for el in range(len(recs[level][row])):
                    #if a module is not close to a X border
                    if el >= mods and el < len(recs[level][row]) - mods:
                        # soft rule:
                        # no opening within n modules on x axes 
                        cond1 = sum(recs[level][row][el - mods : el + mods + 1]) == 2 * mods + 1
                        # no opening within n modules on y axes
                        ymods = 0
                        for i in range(-mods, mods + 1):
                            ymods += recs[level][row + i][el] 
                        cond2 = ymods == 2 * mods + 1
                        if cond1 and cond2:
                            recs[level][row][el] = 0
    return(recs)

## Getting random combination

Introducing Exceptions (gallery and tower)

In [ ]:
recsel = [[[1 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
# Exception 1: Gallery
recsel = get_exceptions(recsel, math.floor(30 / ml) - 1, 0, 0, math.ceil(15 / ml), 100, 2) 
# Exception 2: Tower
recsel = get_exceptions(recsel, math.floor(20 / ml), 0, 8, 100, 100, 100) 
recsel

Removing modules till the built area is below the maximal area

In [ ]:
# This module is the first version of the random_tower function

# recsel = [[[0 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
# flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
# area = sum(flatrecs) * mw * ml
# tollerance = 1.4

# while area < MGF * tollerance:
#     for level in range(len(recsel)):
#         for row in range(len(recsel[level])):
#             for el in range(len(recsel[level][row])):
#                 if gr[level][row][el] + random.random() > 1.0:
#                     recsel[level][row][el] = 1
#             # Exception 1: Gallery
#             recsel = get_exceptions(recsel, math.floor(30 / ml) - 1, 0, 0, math.ceil(15 / ml), 100, 2) 
#             # Exception 2: Tower
#             recsel = get_exceptions(recsel, math.floor(20 / ml), 0, 8, 100, 100, 100)
#             # Cantilever rule
#             recsel = cantilever(recsel)
#             # Compact building rule
#             recsel = compact_building(recsel)
#             # 
#             flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
#             area = sum(flatrecs) * mw * ml
# recsel

## Function to generate and export more random solutions

In [ ]:
def random_tower(experiment, iters, shape, exceptions, MaximalArea, GravityThreshold, GForce, CornerWeights, corners = True, gravityCenters = 4, dim = 2, CantileverRule = True, CompactBuildingRule = True, RoomDepthRule = 'hard', verbose = False):
    
    # Load packages
    import csv
    import datetime
    import random
    
    # INPUTS
            
    # experiment:            name of the experiment, the csv file will be saved under the name
    # iter:                  number of desired solutions
    # shape:                 List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # exceptions:            List wiht all the exceptions, or areas where there shall be no module.
    #                        Each exception is described in a list with [starting X, startingY, starting Z,
    #                        length(X dimension), width (Y dimension), height (Z dimension)]
    # MaximalArea            maximal gross area
    # GravityThreshold:      gravity threshold
    # GForce:                gravity force
    # CornerWeights:         weight of each corner
    # corners                if True the corners are used as center of gravitation, if False random points will be selected
    # gravityCenters         if corners is False, this parameter specifies the number of gravity centers
    # dim                    can be 2 or 3. If corners is False, the graviation can be calculated as 2 or as 3 dimensional
    # CantileverRule:        if 0 the cantilever rule won't be applied, if bigger than 0 the cantilever rule will be applied
    #                        the desired number of steps
    # CompactBuildingRule:   if true the compact building rule is used
    # RoomDepthRule:         "soft" for soft room depht rule, "hard" for hard room depth rule
    # Verbose:               if true prints reports during the execution of the function
    
    # OUTPUT
    
    combinations = [] # List containing all the possible combinations, main output
    
    # where the experiments are being saved
    folder = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/"
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height
    
    # modules on the x axis 
    modx = math.floor(BL / ML)
    # modules on the y axis 
    mody = math.floor(BW / MW)
    # modules on the z axis 
    modz = math.floor(BH / MH)
    
    # maximal number of modules
    mods = math.floor(MaximalArea / (MW * ML))
    if verbose:
        print("Maximal number of modules according to the geometry: ", modx * mody * modz)
        print("Maximal modules according to construction law:", mods)
    # theoretically possible modules after exceptions
    possible = [[[1 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
    # Exception 1: Gallery
    possible = get_exceptions(possible, math.floor(30 / ML) - 1, 0, 0, math.ceil(15 / ML), 100, 2) 
    # Exception 2: Tower
    possible = get_exceptions(possible, math.floor(27 / ML), 0, 8, 100, 100, 100)
    if verbose:
        print("Possible Modules after exceptions: ", sum([item for sublist in [item for sublist in possible for item in sublist] for item in sublist]))
        print("*" * 30)
    
    # coordinates of the module centers
    recs = [[[1 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
    coords = get_center(recs, ML, MW, MH)
    
    # gravity cohefficient
    gr = gravitycoehfficients(coords = coords, gforce = GForce, weights = CornerWeights, corners = corners, gravityCenters = gravityCenters, dim = dim)
    grflat = [item for sublist in [item for sublist in gr for item in sublist] for item in sublist]
    plt.hist(grflat, bins = 50, alpha = 0.2)
    
    # main loop
    for i in range(iters):
        
        # gravity cohefficient
        gr = gravitycoehfficients(coords = coords, gforce = GForce, weights = CornerWeights, corners = corners, gravityCenters = gravityCenters, dim = dim)
        grflat = [item for sublist in [item for sublist in gr for item in sublist] for item in sublist]
        plt.hist(grflat, bins = 50)
        
        recsel = [[[0 for x in repeat(None, modx)] for x in repeat(None, mody)] for x in repeat(None, modz)]
        flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
        area = sum(flatrecs) * mw * ml
        # tolerance would limit the number of modules picked; not necessary anymore, because of index shuffling
        tolerance = 2.3
        
        # Shuffling the selectors in order to pick different modules
        zsel = list(range(len(recs)))
        random.shuffle(zsel)
        ysel = list(range(len(recs[0])))
        random.shuffle(ysel)
        xsel = list(range(len(recs[0][0])))
        random.shuffle(xsel)
        
        # iteration counter
        iterations = -1
        
        while sum(flatrecs) < mods:
            
            # Gravity Threshold as mobile target, in order to obtain solutions
            iterations += 1
            gt = GravityThreshold - iterations / 1000
            
            for level in zsel: #range(len(recsel)):
                for row in ysel: #range(len(recsel[level])):
                    for el in xsel: #range(len(recsel[level][row])):
                        if (gr[level][row][el] + (random.random() / 10) > gt): # and sum(flatrecs) < mods * tolerance):
                            recsel[level][row][el] = 1
                            # Updating stop condition
                            flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
            if verbose:
                print("Iteration: ", i, "/", iters)
                print("before rules: ", sum(flatrecs))
            # Applying exceptions
            if len(exceptions) > 0:
                for exc in exceptions:
                    recsel = get_exceptions(recsel, exc[0], exc[1], exc[2], exc[3], exc[4], exc[5])
            # # Exception 1: Gallery
            # recsel = get_exceptions(recsel, math.floor(30 / ML) - 1, 0, 0, math.ceil(15 / ML), 100, 2) 
            # # Exception 2: Tower
            # recsel = get_exceptions(recsel, math.floor(27 / ML), 0, 8, 100, 100, 100)
            if verbose:
                print("after exceptions:", sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]))
            # Cantilever rule
            if CantileverRule > 0:
                recsel = cantilever(recsel, steps = CantileverRule)
                numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                if verbose:
                    print("after cantilever:", numberselected, "/", mods)
            # Compact building rule
            if CompactBuildingRule:
                recsel = compact_building(recsel)
                numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                if verbose:
                    print("after compact building:", numberselected, "/", mods)
            # Hard Room depth rule
            if RoomDepthRule == "hard":
                recsel = roomdepth(recsel, math.ceil(21 / ((ML + MW) / 2)))
                numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                if verbose:
                    print("after hard room depth:", numberselected, "/", mods)
            # Soft Room depth rule
            if RoomDepthRule == "soft":
                recsel = softroomdepth(recsel, 3)
                numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                if verbose:
                    print("after soft room depth:", numberselected, "/", mods)
            # Updating stop condition
            flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
            area = sum(flatrecs) * mw * ml
            if verbose:
                print("_____________________")
        combinations.append(recsel)
        
        # Exporting centers of rectangles
        centers = get_center(recsel, ML, MW, MH)
        flatcenters = [item for sublist in [item for sublist in centers for item in sublist] for item in sublist]
        filename = folder + experiment + '_' + str(i) + '.csv'
        with open(filename, mode = 'w', newline = '') as csv_file:
            writer = csv.writer(csv_file, delimiter = ",", quoting = csv.QUOTE_MINIMAL)
            for row in flatcenters:
                writer.writerow(row)
        
        # Report experiment specifications:
        repfile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/_report.csv"
        with open(repfile, 'r') as f:
            reader = csv.reader(f, delimiter = ';')
            report = list(reader)
        
        today = datetime.date.today().strftime("%d/%m/%y")
        
        report.append([today, experiment, iters, shape, GravityThreshold, GForce, CornerWeights, CantileverRule, CompactBuildingRule, RoomDepthRule])
        
        with open(repfile, 'w', newline = '') as f:
            writer = csv.writer(f, delimiter = ";")
            writer.writerows(report)
        
        
    return(combinations)

In [ ]:
# t = random_tower('negativegravity_softroom', iters = 30, shape = [65, 55, 100, 3, 3, 3], MaximalArea = 34500 / 0.9, GravityThreshold = 0.05, GForce = 0.1, CornerWeights = [-0.2, 0.5, 0.3, 0.4], CantileverRule = True, CompactBuildingRule = True, RoomDepthRule = 'soft')

### Block and Tower separate

In [ ]:
# t = random_tower('Cant_16AGF_RandomNegG_1P3D_hardroom_RD24_block', iters = 30, shape = [65, 55, 23, 3, 3, 3], exceptions = [[10, 0, 0, 5, 100, 2]], MaximalArea = 16000 / 0.9, GravityThreshold = -0.05, GForce = 0.1, CornerWeights = [-1, 1, 0.4, 0.4], corners = False, gravityCenters = 1, dim = 3, CantileverRule = 0, CompactBuildingRule = True, RoomDepthRule = 'hard', verbose = True)

In [ ]:
t = random_tower('Cant_16AGF_RandomNegG_1P3D_hardroom_RD24_tower', iters = 30, shape = [24, 55, 100 - 23, 3, 3, 3], exceptions = [], MaximalArea = 17000 / 0.9, GravityThreshold =  0.1, GForce = 0.15, CornerWeights = [-1, 1, 0.4, 0.4], corners = False, gravityCenters = 1, dim = 3, CantileverRule = 1, CompactBuildingRule = True, RoomDepthRule = 'hard', verbose = True) 

## Get the cluster perimeter from the combinations

The function will produce a list of perimetral points

### Find Clusters

In [ ]:
def clusters(shape, combinations):
    
    # INPUT 
    # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # combinations:   result from random_tower function
    
    # OUTPUT 
    
    comb_clusters = [] # List of combinations of clusters
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height

    for comb in combinations:
        c = get_center(comb, ML, MW, MH)
        clusters = []

        for level in range(len(c)):
            centers_in_lev = [item for sublist in c[level] for item in sublist]
            while len(centers_in_lev) > 0:
                newcluster = [centers_in_lev[0]]
                candidates = [centers_in_lev[0]]
                centers_in_lev.pop(0)
                while len(candidates) > 0:
                    candidate = candidates[0]
                    candidates.pop(0)
                    cX = candidate[0]
                    cY = candidate[1]
                    for totest in centers_in_lev:
                        testX = totest[0]
                        testY = totest[1]
                        cond1 = (cX - testX == 0) and (abs(cY - testY) == MW)
                        cond2 = (abs(cX - testX) == ML) and (cY - testY == 0)
                        if (cond1 or cond2):
                            newcluster.append(totest)
                            candidates.append(totest)
                            centers_in_lev.remove(totest)
                clusters.append(newcluster)
                
                # Export each cluster in a separate csv
                # clusterfile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/cluster_"
                # for cl in range(len(clusters)):
                #     cl0 = clusters[cl]
                #     file = clusterfile + str(cl) + ".csv"
                #     with open(file, 'w', newline = '') as f:
                #         writer = csv.writer(f, delimiter = ",")
                #         writer.writerows(cl0)
        comb_clusters.append(clusters)
    return(comb_clusters)

In [ ]:
cl = clusters(shape = [65, 55, 23, 3, 3, 3], combinations = t)
cl

## Find  gaps within a cluster

In [ ]:
def find_gaps(shape, clusters):
    
    # INPUT 
    # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # clusters:       result from clusters function with this structure [combination [cluster [point[X, Y, Z]]]]
    
    # OUTPUT 
    
    cluster_gaps = [] # List of combinations of clusters  with this structure [combination, [cluster [gap [point [X, Y, Z]]]]]
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height
    
    for combination in clusters:
        for clus in combination:
            # 1: getting bounding box for the cluster
            MaxX = -9e20
            MinX = +9e20
            MaxY = -9e20
            MinY = +9e20
            Z = clus[0][2]
            for point in clus:
                if point[0] > MaxX:
                    MaxX = point[0]
                if point[0] < MinX:
                    MinX = point[0]
                if point[1] > MaxY:
                    MaxY = point[1]
                if point[1] < MinY:
                    MinY = point[1]
            # 2: getting all the module centers not in the cluster list
            # mods = len(clus) 
            # modsX = (MaxX - MinX) / ML
            # modsY = (MaxY - MinY) / MW
            candidates = [[x, y, Z] for x in np.arange(MinX, MaxX, ML) for y in np.arange(MinY, MaxY, MW)]
            gap_centers = [c for c in candidates if c not in clus]
            # 3 cluster the module centers - based on clusters function
            gaps_in_cluster = []
            while len(gap_centers) > 0:
                newcluster = [gap_centers[0]]
                candidates = [gap_centers[0]]
                gap_centers.pop(0)
                while len(candidates) > 0:
                    candidate = candidates[0]
                    candidates.pop(0)
                    cX = candidate[0]
                    cY = candidate[1]
                    for totest in gap_centers:
                        testX = totest[0]
                        testY = totest[1]
                        cond1 = (cX - testX == 0) and (abs(cY - testY) == MW)
                        cond2 = (abs(cX - testX) == ML) and (cY - testY == 0)
                        if (cond1 or cond2):
                            newcluster.append(totest)
                            candidates.append(totest)
                            gap_centers.remove(totest)
                if len(newcluster) > 0 :
                    gaps_in_cluster.append(newcluster)
        cluster_gaps.append(gaps_in_cluster)
        
    return(cluster_gaps)
            

In [ ]:
find_gaps(shape = [65, 55, 23, 3, 3, 3], clusters = cl)

## Get only modules on the edge

In [ ]:
def get_perimetral_modules(shape, clusters):
    
    # INPUT 
    # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # clusters:       result from clusters function
    
    # OUTPUT 
    
    perimetral = [] # List of combinations of clusters
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height
    
    for combination in clusters:
        combination_perimeters = []
        for clus in combination:
            clus_perimeter = []
            for center in clus:
                # moving the point in the two directions
                up = [center[0], center[1] + MW, center[2]]
                down = [center[0], center[1] - MW, center[2]]
                right = [center[0] + ML, center[1], center[2]]
                left = [center[0] - ML, center[1], center[2]]
                if not (up in clus and down in clus and right in clus and left in clus):
                    clus_perimeter.append(center)
            combination_perimeters.append(clus_perimeter)
        perimetral.append(combination_perimeters)
    
    # Export each cluster in a separate csv
    # perifile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/cluster_perimeter_"
    # for pe in range(len(perimetral[-1])):
    #     pe0 = perimetral[-1][pe]
    #     file = perifile + str(pe) + ".csv"
    #     with open(file, 'w', newline = '') as f:
    #         writer = csv.writer(f, delimiter = ",")
    #         writer.writerows(pe0)
    
    return(perimetral)

In [ ]:
pm = get_perimetral_modules(shape = [65, 55, 23, 3, 3, 3], clusters = cl)
pm

### Get all points of perimetral modules one time

In [ ]:
def perimetral_modules_points(shape, perimetral):
    
    # INPUT 
    # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # perimetral:     result from get_perimetral_modules function
    
    # OUTPUT 
    
    peripoints = [] # List corners of modules. Point will be copied on the list just once 
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height
    
    perimetral_points = []
    
    for configuration in perimetral:
        comb_points = []
        for cluster in configuration:
            point_list = []
            for point in cluster:
                topleft = [point[0] - ML / 2, point[1] + MW / 2, point[2]]
                topright = [point[0] + ML / 2, point[1] + MW / 2, point[2]]
                bottomright = [point[0] + ML / 2, point[1] - MW / 2, point[2]]
                bottomleft = [point[0] - ML / 2, point[1] - MW / 2, point[2]]
                for p in [topleft, topright, bottomleft, bottomright]:
                    if p not in point_list:
                        point_list.append(p)
            comb_points.append(point_list)
        perimetral_points.append(comb_points)
    
    return(perimetral_points)

In [ ]:
pp = perimetral_modules_points(shape = [65, 55, 23, 3, 3, 3], perimetral = pm)
pp

### Get Cluster perimeter

In [ ]:
### def perimeter_points(experiment, shape, perimetral, centers, verbose = False):
###     
###     # This function takes the corner points of the perimetral modules and returns a list of perimeters
###     
###     # INPUT 
###     # experiment:     name of the experiment and of the output files
###     # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
###     # perimetral:     result from perimetral_modules_points function (list of all points on the perimetral modules by cluster)
###     # centers:        result of the clusters function (list of all the module center within each cluster)
###     # verbose:        if true prints function parameters on the differente stage for debugging
###     
###     # OUTPUT 
###     
###     peripoints = [] # List of combinations of clusters
###     
###     # geometric parameters
###     BL = shape[0] # building length
###     BW = shape[1] # building width
###     BH = shape[2] # building height
###     ML = shape[3] # module length
###     MW = shape[4] # module width
###     MH = shape[5] # module height
###     
###     for configuration in perimetral:
###         ind0 = perimetral.index(configuration)
###         # list of all the cluster perimeters within a configuration
###         cluster_perimeters = []
###         for cluster in configuration:
###             if verbose:
###                 print("*" * int(1e2))
###             ind1 = configuration.index(cluster)
###             # list of module centers in the cluster
###             clusterCenters = centers[ind0][ind1] 
###             remaining_points = cluster[:]
###             # list of all points in the perimeter
###             perimeter_list = []
###             # all x values within the cluster
###             Xs = []
###             # all y values within the cluster
###             Ys = []
###             Z = remaining_points[0][2]
###             for point in cluster:
###                 Xs.append(point[0])
###                 Ys.append(point[1])
###             Xs.sort()
###             Ys.sort()
###             # List of all the points by Y coordinate
###             pointsByY = []
###             for Y in Ys:
###                 to_add = []
###                 for point in cluster:
###                     if point[1] == Y:
###                         to_add.append(point)
###                 pointsByY.append(to_add)
###             # The starting point will be the one with the smallest X within the ones with the smallest Y
###             downX = []
###             for point in pointsByY[0]:
###                 downX.append(point[0])
###             minX = min(downX)
###             for point in pointsByY[0]:
###                 if point[0] == minX:
###                     startingPoint = point
###                     pointsByY[0].remove(point)
###                     perimeter_list.append(startingPoint)
###                     remaining_points.remove(startingPoint)
###             # iterative process to get the perimeter chain
###             if verbose:
###                 print("points in cluster : ", len(cluster))
###                 print("starting point: ", startingPoint)
###             while perimeter_list[-1] != startingPoint or len(perimeter_list) == 1:
###                 p0 = perimeter_list[-1]
###                 up = [p0[0], p0[1] + MW, p0[2]]
###                 down = [p0[0], p0[1] - MW, p0[2]]
###                 right = [p0[0] + ML, p0[1], p0[2]]
###                 left = [p0[0] - ML, p0[1], p0[2]]
###                 if verbose:
###                     print("last point: ", p0)
###                     print("surrounding point: ", up, down, left, right)
###                 # check all the points close to p0 within the list of remaining points
###                 candidates = [value for value in remaining_points if value in [up, down, left, right]]
###                 if len(candidates) > 0:
###                     if verbose:
###                         print("candidates: ", candidates)
###                     for can in candidates:
###                         # if p0 (previous perimeter point) and candidate share the same Y value
###                         if (can == up or can == down):
###                             # condition: in order the point to lay on the perimeter 
###                             # there should be a single module center between the two points 
###                             # on the right or left
###                             possibleCenterR = [((p0[0] + can[0]) / 2) + ML / 2, ((p0[1] + can[1]) / 2), Z]
###                             possibleCenterL = [((p0[0] + can[0]) / 2) - ML / 2, ((p0[1] + can[1]) / 2), Z]
###                             # find out if the candidate complies with the perimeter condition 
###                             # (separate module centers in the figure from centers not in the figure)
###                             # and the direction and way condition
###                             if p0[1] < can[1]:
###                                 cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
###                             if p0[1] > can[1]:
###                                 cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
###                             if verbose:
###                                 print("candidate UP/DOWN: ", can)
###                                 print("point 0: ", p0)
###                                 print("possible centers : ", possibleCenterR, possibleCenterL)
###                                 print("cluster centers: ", clusterCenters)
###                                 print("conditions respected:", cond)
###                             if cond:
###                                 perimeter_list.append(can)
###                                 remaining_points.remove(can)
###                                 if verbose:
###                                     print("new Point added", perimeter_list)
###                                     print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                     print("-" * int(1e1))
###                                 break
###                         
###                         # if p0 (previous perimeter point) and candidate share the same X value
###                         else: 
###                             if (can == right or can == left ):
###                                 # condition: in order the point to lay on the perimeter 
###                                 # there should be a module center between the two points 
###                                 # on the top or bottom
###                                 possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + MW / 2, Z]
###                                 possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - MW / 2, Z]
###                                 # find out if the candidate complies with the perimeter condition 
###                                 # (separate module centers in the figure from centers not in the figure)
###                                 # and the direction and way condition
###                                 if p0[0] > can[0]:
###                                     cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
###                                 if p0[0] < can[0]:
###                                     cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
###                                 if verbose:
###                                     print("candidate LEFT/RIGHT: ", can)
###                                     print("point 0: ", p0)
###                                     print("possible centers : ", possibleCenterT, possibleCenterB)
###                                     print("cluster centers: ", clusterCenters)
###                                     print("conditions respected:", cond)
###                                 if cond:
###                                     perimeter_list.append(can)
###                                     remaining_points.remove(can)
###                                     if verbose:
###                                         print("new Point added", perimeter_list)
###                                         print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                         print("-" * int(1e1))
###                                     break
###                     # after trying all candidates, if no candidate was added to the 
###                     # perimeter list, the algorithm verifies the possibility
###                     # to close the perimeter
###                     if startingPoint in [up, down, left, right] and p0 == perimeter_list[-1]:
###                         if verbose:
###                             print("closing the circle")
###                         perimeter_list.append(startingPoint)
###                         break
###                     # some configurations may lead to vertex overlapping.
###                     # to solve this, if no other candidate is found, vertices already
###                     # in the perimeter list can be reconsidered as candidates
###                     if p0 == perimeter_list[-1]:
###                         # last point to be excluded to avoid infinte loop
###                         candidates = [value for value in perimeter_list[:-1] if value in [up, down, left, right]]
###                         # print("candidates: ", candidates)
###                         for can in candidates:
###                             # if p0 (previous perimeter point) and candidate share the same Y value
###                             if (can == up or can == down):
###                                 possibleCenterR = [((p0[0] + can[0]) / 2) + ML / 2, ((p0[1] + can[1]) / 2), Z]
###                                 possibleCenterL = [((p0[0] + can[0]) / 2) - ML / 2, ((p0[1] + can[1]) / 2), Z]
###                                 if p0[1] < can[1]:
###                                     cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
###                                 if p0[1] > can[1]:
###                                     cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
###                                 if cond:
###                                     perimeter_list.append(can)
###                                     if verbose:
###                                         print("old Point added", perimeter_list)
###                                         print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                         print("-" * int(1e1))
###                                     break
###                             # if p0 (previous perimeter point) and candidate share the same X value
###                             else: 
###                                 if (can == right or can == left ):
###                                     possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + MW / 2, Z]
###                                     possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - MW / 2, Z]
###                                     if p0[0] > can[0]:
###                                         cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
###                                     if p0[0] < can[0]:
###                                         cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
###                                     # print("starting point: ", p0)
###                                     # print("candidate: ", can)
###                                     # print("Condition respected:", cond)
###                                     if cond:
###                                         perimeter_list.append(can)
###                                         if verbose:
###                                             print("old Point added", perimeter_list)
###                                             print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                             print("-" * int(1e1))
###                                         break
###                 # if there are no candidates and the last point is close to 
###                 # the starting point, the polygon is closed
###                 else:
###                     if startingPoint in [up, down, left, right]:
###                         if verbose:
###                             print("closing the circle")
###                         perimeter_list.append(startingPoint)
###                         break
###                     else:
###                         # some configurations may lead to vertex overlapping.
###                         # to solve this, if no other candidate is found, vertices already
###                         # in the perimeter list can be reconsidered as candidates
###                         if p0 == perimeter_list[-1]:
###                             # last point to be excluded to avoid infinte loop
###                             candidates = [value for value in perimeter_list[:-1] if value in [up, down, left, right]]
###                             for can in candidates:
###                                 # if p0 (previous perimeter point) and candidate share the same Y value
###                                 if (can == up or can == down):
###                                     possibleCenterR = [((p0[0] + can[0]) / 2) + ML / 2, ((p0[1] + can[1]) / 2), Z]
###                                     possibleCenterL = [((p0[0] + can[0]) / 2) - ML / 2, ((p0[1] + can[1]) / 2), Z]
###                                     if p0[1] < can[1]:
###                                         cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
###                                     if p0[1] > can[1]:
###                                         cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
###                                     if cond:
###                                         perimeter_list.append(can)
###                                         if verbose:
###                                             print("old Point added", perimeter_list)
###                                             print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                             print("-" * int(1e1))
###                                         break
###                                 # if p0 (previous perimeter point) and candidate share the same X value
###                                 else: 
###                                     if (can == right or can == left ):
###                                         possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + MW / 2, Z]
###                                         possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - MW / 2, Z]
###                                         if p0[0] > can[0]:
###                                             cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
###                                         if p0[0] < can[0]:
###                                             cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
###                                         if cond:
###                                             perimeter_list.append(can)
###                                             if verbose:
###                                                 print("old Point added", perimeter_list)
###                                                 print("remaining points: ", len(remaining_points), "/", len(cluster))
###                                                 print("-" * int(1e1))
###                                             break
###                 
###             # removing reduntant starting point
###             perimeter_list.pop(-1)
###             cluster_perimeters.append(perimeter_list)
###         peripoints.append(cluster_perimeters)
###     
###     # Export each point cluster in a separate csv
###     perifile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/" + experiment + "_peri_"
###     
###     for config in peripoints:
###         ind = peripoints.index(config)
###         periflat = []
###         for peri in config:
###             for point in peri:
###                 periflat.append(point)
###             periflat.append(["next cluster", "", ""])
###         # pe0 = peripoints[pe]
###         file = perifile + str(ind) + ".csv"
###         with open(file, 'w', newline = '') as f:
###             writer = csv.writer(f, delimiter = ",")
###             writer.writerows(periflat)
###     
###     return(peripoints)

In [ ]:
def perimeter_points(experiment, shape, perimetral, centers, verbose = False):
    
    # This function takes the corner points of the perimetral modules and returns a list of perimeters
    
    # INPUT 
    # experiment:     name of the experiment and of the output files
    # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # perimetral:     result from perimetral_modules_points function (list of all points on the perimetral modules by cluster)
    # centers:        result of the clusters function (list of all the module center within each cluster)
    # verbose:        if true prints function parameters on the differente stage for debugging
    
    # OUTPUT 
    
    peripoints = [] # List of combinations of clusters
    
    # geometric parameters
    BL = shape[0] # building length
    BW = shape[1] # building width
    BH = shape[2] # building height
    ML = shape[3] # module length
    MW = shape[4] # module width
    MH = shape[5] # module height
    
    for configuration in perimetral:
        ind0 = perimetral.index(configuration)
        # list of all the cluster perimeters within a configuration
        cluster_perimeters = []
        for cluster in configuration:
            if verbose:
                print("*" * int(1e2))
            ind1 = configuration.index(cluster)
            # list of module centers in the cluster
            clusterCenters = centers[ind0][ind1] 
            remaining_points = cluster[:]
            # list of all points in the perimeter
            perimeter_list = []
            # all x values within the cluster
            Xs = []
            # all y values within the cluster
            Ys = []
            Z = remaining_points[0][2]
            for point in cluster:
                Xs.append(point[0])
                Ys.append(point[1])
            Xs.sort()
            Ys.sort()
            # List of all the points by Y coordinate
            pointsByY = []
            for Y in Ys:
                to_add = []
                for point in cluster:
                    if point[1] == Y:
                        to_add.append(point)
                pointsByY.append(to_add)
            # The starting point will be the one with the smallest X within the ones with the smallest Y
            downX = []
            for point in pointsByY[0]:
                downX.append(point[0])
            minX = min(downX)
            for point in pointsByY[0]:
                if point[0] == minX:
                    startingPoint = point
                    pointsByY[0].remove(point)
                    perimeter_list.append(startingPoint)
                    remaining_points.remove(startingPoint)
            # iterative process to get the perimeter chain
            if verbose:
                print("points in cluster : ", len(cluster))
                print("starting point: ", startingPoint)
            while perimeter_list[-1] != startingPoint or len(perimeter_list) == 1:
                p0 = perimeter_list[-1]
                up = [p0[0], p0[1] + MW, p0[2]]
                down = [p0[0], p0[1] - MW, p0[2]]
                right = [p0[0] + ML, p0[1], p0[2]]
                left = [p0[0] - ML, p0[1], p0[2]]
                if verbose:
                    print("last point: ", p0)
                    print("surrounding point: ", up, down, left, right)
                # check all the points close to p0 within the list of remaining points
                candidates = [value for value in remaining_points if value in [up, down, left, right]]
                if verbose:
                    print("candidates: ", candidates)
                for can in candidates:
                    # if p0 (previous perimeter point) and candidate share the same Y value
                    if (can == up or can == down):
                        # condition: in order the point to lay on the perimeter 
                        # there should be a single module center between the two points 
                        # on the right or left
                        possibleCenterR = [((p0[0] + can[0]) / 2) + ML / 2, ((p0[1] + can[1]) / 2), Z]
                        possibleCenterL = [((p0[0] + can[0]) / 2) - ML / 2, ((p0[1] + can[1]) / 2), Z]
                        # find out if the candidate complies with the perimeter condition 
                        # (separate module centers in the figure from centers not in the figure)
                        # and the direction and way condition
                        if p0[1] < can[1]:
                            cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
                        if p0[1] > can[1]:
                            cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
                        if verbose:
                            print("candidate UP/DOWN: ", can)
                            print("point 0: ", p0)
                            print("possible centers : ", possibleCenterR, possibleCenterL)
                            print("cluster centers: ", clusterCenters)
                            print("conditions respected:", cond)
                        if cond:
                            perimeter_list.append(can)
                            remaining_points.remove(can)
                            if verbose:
                                print("new Point added", perimeter_list)
                                print("remaining points: ", len(remaining_points), "/", len(cluster))
                                print("-" * int(1e1))
                            break
                    
                    # if p0 (previous perimeter point) and candidate share the same X value
                    else: 
                        if (can == right or can == left ):
                            # condition: in order the point to lay on the perimeter 
                            # there should be a module center between the two points 
                            # on the top or bottom
                            possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + MW / 2, Z]
                            possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - MW / 2, Z]
                            # find out if the candidate complies with the perimeter condition 
                            # (separate module centers in the figure from centers not in the figure)
                            # and the direction and way condition
                            if p0[0] > can[0]:
                                cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
                            if p0[0] < can[0]:
                                cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
                            if verbose:
                                print("candidate LEFT/RIGHT: ", can)
                                print("point 0: ", p0)
                                print("possible centers : ", possibleCenterT, possibleCenterB)
                                print("cluster centers: ", clusterCenters)
                                print("conditions respected:", cond)
                            if cond:
                                perimeter_list.append(can)
                                remaining_points.remove(can)
                                if verbose:
                                    print("new Point added", perimeter_list)
                                    print("remaining points: ", len(remaining_points), "/", len(cluster))
                                    print("-" * int(1e1))
                                break
                # after trying all candidates, if no candidate was added to the 
                # perimeter list, the algorithm verifies the possibility
                # to close the perimeter
                if startingPoint in [up, down, left, right] and p0 == perimeter_list[-1]:
                    if verbose:
                        print("closing the circle")
                    perimeter_list.append(startingPoint)
                    break
                # some configurations may lead to vertex overlapping.
                # to solve this, if no other candidate is found, vertices already
                # in the perimeter list can be reconsidered as candidates
                if p0 == perimeter_list[-1]:
                    # last point to be excluded to avoid infinte loop
                    candidates = [value for value in perimeter_list[:-1] if value in [up, down, left, right]]
                    # print("candidates: ", candidates)
                    for can in candidates:
                        # if p0 (previous perimeter point) and candidate share the same Y value
                        if (can == up or can == down):
                            possibleCenterR = [((p0[0] + can[0]) / 2) + ML / 2, ((p0[1] + can[1]) / 2), Z]
                            possibleCenterL = [((p0[0] + can[0]) / 2) - ML / 2, ((p0[1] + can[1]) / 2), Z]
                            if p0[1] < can[1]:
                                cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
                            if p0[1] > can[1]:
                                cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
                            if cond:
                                perimeter_list.append(can)
                                if verbose:
                                    print("old Point added", perimeter_list)
                                    print("remaining points: ", len(remaining_points), "/", len(cluster))
                                    print("-" * int(1e1))
                                break
                        # if p0 (previous perimeter point) and candidate share the same X value
                        else: 
                            if (can == right or can == left ):
                                possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + MW / 2, Z]
                                possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - MW / 2, Z]
                                if p0[0] > can[0]:
                                    cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
                                if p0[0] < can[0]:
                                    cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
                                # print("starting point: ", p0)
                                # print("candidate: ", can)
                                # print("Condition respected:", cond)
                                if cond:
                                    perimeter_list.append(can)
                                    if verbose:
                                        print("old Point added", perimeter_list)
                                        print("remaining points: ", len(remaining_points), "/", len(cluster))
                                        print("-" * int(1e1))
                                    break
            # removing reduntant starting point
            perimeter_list.pop(-1)
            cluster_perimeters.append(perimeter_list)
        peripoints.append(cluster_perimeters)
    
    # Export each point cluster in a separate csv
    perifile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/" + experiment + "_peri_"
    
    for config in peripoints:
        ind = peripoints.index(config)
        periflat = []
        for peri in config:
            for point in peri:
                periflat.append(point)
            periflat.append(["next cluster", "", ""])
        # pe0 = peripoints[pe]
        file = perifile + str(ind) + ".csv"
        with open(file, 'w', newline = '') as f:
            writer = csv.writer(f, delimiter = ",")
            writer.writerows(periflat)
    
    return(peripoints)

In [ ]:
pe = perimeter_points(experiment = "test_perimeter", shape = [65, 55, 23, 3, 3, 3], perimetral = pp, centers = cl, verbose = True)

In [ ]:
pe

In [ ]:
t

### Debugging the perimeter_points rule

In [ ]:
# ind = [12, 34]
# test = [[pp[ind[0]][ind[1]][:]]]
# testCE = [[cl[ind[0]][ind[1]][:]]]
# pe = perimeter_points(experiment = "test_perimeter", shape = [65, 55, 23, 3, 3, 3], perimetral = test, centers = testCE, verbose = True)
# Xs = []
# Ys = []
# for point in pe[0][0]:
#     Xs.append(point[0])
#     Ys.append(point[1])
# Xs.append(pe[0][0][0][0])
# Ys.append(pe[0][0][0][1])
# xc = []
# yc = []
# for p in testCE[0][0]:
#     xc.append(p[0])
#     yc.append(p[1])
# plt.plot(Xs, Ys, marker = "*", markersize = 15)
# plt.scatter(xc, yc, color = "red", marker = "+", s = 100);

In [ ]:
# ind0 = [25 for i in range(25)]
# ind1 = list(range(25))
# for i in range(25):
#     print("INDEX:", i)
#     print("*" * int(1e2))
#     print("-" * int(1e2))
#     print("*" * int(1e2))
#     test = [[pp[ind0[i]][ind1[i]][:]]]
#     testCE = [[cl[ind0[i]][ind1[i]][:]]]
#     pe = perimeter_points(experiment = "test_perimeter", shape = [65, 55, 23, 3, 3, 3], perimetral = test, centers = testCE)
#     Xs = []
#     Ys = []
#     for point in pe[0][0]:
#         Xs.append(point[0])
#         Ys.append(point[1])
#     Xs.append(pe[0][0][0][0])
#     Ys.append(pe[0][0][0][1])
#     xc = []
#     yc = []
#     for p in testCE[0][0]:
#         xc.append(p[0])
#         yc.append(p[1])
#     plt.plot(Xs, Ys, marker = "*", markersize = 15)
#     plt.scatter(xc, yc, color = "red", marker = "+", s = 100);

In [ ]:
# This block visualize each cluster as scatter

# ind = [12, 34]
# test = [[pp[ind[0]][ind[1]][:]]]
# testCE = [[pm[ind[0]][ind[1]][:]]]
# testCL = [[cl[ind[0]][ind[1]][:]]]
# 
# xc = []
# yc = []
# xs = []
# ys = []
# for p in testCE[0][0]:
#     xs.append(p[0])
#     ys.append(p[1])
# for p in testCL[0][0]:
#     xc.append(p[0])
#     yc.append(p[1])
# 
# plt.scatter(xc, yc, color = 'red')
# plt.scatter(xs, ys);

# Putting all together

In [ ]:
# putting all together in a single workflow:
def gravity_random_generator(params):
    
    # INPUT 
    
    # in the params dictionary the following elements need to be defined 
    # experiment, iters, shape, exceptions, MaximalArea, GravityThreshold, GForce, CornerWeights, corners, gravityCenters, dim, CantileverRule, CompactBuildingRule, RoomDepthRule, verbose
    
    # experiment:            name of the experiment, the csv file will be saved under the name
    # iter:                  number of desired solutions
    # shape:                 List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
    # exceptions:            List wiht all the exceptions, or areas where there shall be no module.
    #                        Each exception is described in a list with [starting X, startingY, starting Z,
    #                        length(X dimension), width (Y dimension), height (Z dimension)]
    # MaximalArea            maximal gross area
    # GravityThreshold:      gravity threshold
    # GForce:                gravity force
    # CornerWeights:         weight of each corner
    # corners                if True the corners are used as center of gravitation, if False random points will be selected
    # gravityCenters         if corners is False, this parameter specifies the number of gravity centers
    # dim                    can be 2 or 3. If corners is False, the graviation can be calculated as 2 or as 3 dimensional
    # CantileverRule:        if 0 the cantilever rule won't be applied, if bigger than 0 the cantilever rule will be applied
    #                        the desired number of steps
    # CompactBuildingRule:   if true the compact building rule is used
    # RoomDepthRule:         "soft" for soft room depht rule, "hard" for hard room depth rule
    # Verbose:               if true prints reports during the execution of the function
    
    # 1 - Run random_tower to generate combinations of modules
    t = random_tower(params["experiment"], params["iters"], params["shape"], params["exceptions"], params["MaximalArea"], params["GravityThreshold"], params["GForce"], params["CornerWeights"], params["corners"], params["gravityCenters"], params["dim"], params["CantileverRule"], params["CompactBuildingRule"], params["RoomDepthRule"], params["verbose"])
    print("random configurations generated")
    # 2 - find clusters
    cl = clusters(shape = params["shape"], combinations = t)
    print ("cluster detected")
    # 3 - get clusters on the perimeter
    pm = get_perimetral_modules(shape = params["shape"], clusters = cl)
    print("perimeter modules detected")
    # 4 - get coordinates of cluster point on the perimeter
    pp = perimetral_modules_points(shape = params["shape"], perimetral = pm)
    print("list of point in the perimetral modules complete")
    # 5 - get perimeter points
    peB = perimeter_points(experiment = params["experiment"], shape = params["shape"], perimetral = pp, centers = cl)
    print("session complete: coordinates of the cluster saved")
    return(peB)

### Experiment I: 

full area (34500 m²), hard room depth rule, negative gravity, one center, 3-dimensional gravity

In [ ]:
paramsB01 = {"experiment": "17.5MA_NG_1C_3D_3Ca_rdH_block", 
          "iters": 30, 
          "shape": [65, 55, 23, 3, 3, 3], 
          "exceptions": [[10, 0, 0, 5, 100, 2]], 
          "MaximalArea": 17500 / 0.9, 
          "GravityThreshold": -0.05, 
          "GForce": 0.1, 
          "CornerWeights": [-1, 1, 0.4, 0.4], 
          "corners": False, 
          "gravityCenters": 1, 
          "dim": 3, 
          "CantileverRule": 3, 
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}

E01B = gravity_random_generator(paramsB01)

In [ ]:
paramsT01 = {"experiment": "17.5MA_NG_1C_3D_3Ca_rdH_tower", 
          "iters": 30, 
          "shape": [24, 55, 100 - 23, 3, 3, 3], 
          "exceptions": [], 
          "MaximalArea": 17000 / 0.9, 
          "GravityThreshold": 0.1, 
          "GForce": 0.1, 
          "CornerWeights": [-1, 1, 0.4, 0.4], 
          "corners": False, 
          "gravityCenters": 1, 
          "dim": 3, 
          "CantileverRule": 1, 
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": False}
E01T = gravity_random_generator(paramsT01)

### Experiment II:

reduced block area (16500 + 17000 m²), hard room depth rule, positive gravity, 4 centers, 3-dimensional gravity

In [ ]:
paramsB02 = {"experiment": "33.5MA_PG_4C_3D_3Ca_rdH_block", 
          "iters": 30, 
          "shape": [65, 55, 23, 3, 3, 3], 
          "exceptions": [[10, 0, 0, 5, 100, 2]], 
          "MaximalArea": 16500 / 0.9, 
          "GravityThreshold": 0.2, 
          "GForce": 0.1, 
          "CornerWeights": [1, 1, 0.4, 0.4], 
          "corners": False,
          "gravityCenters": 4,
          "dim": 3,
          "CantileverRule": 3,
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}

E02B = gravity_random_generator(paramsB02)

In [ ]:
paramsT02 = {"experiment": "33.5MA_PG_4C_3D_3Ca_rdH_tower", 
          "iters": 30, 
          "shape": [24, 55, 100 - 23, 3, 3, 3], 
          "exceptions": [], 
          "MaximalArea": 17000 / 0.9, 
          "GravityThreshold": 0.15, 
          "GForce": 0.2, 
          "CornerWeights": [1, 1, 0.4, 0.4], 
          "corners": False, 
          "gravityCenters": 4, 
          "dim": 3, 
          "CantileverRule": 1, 
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}
E02T = gravity_random_generator(paramsT02)

### Experiment 3

reduced block area (16500 + 17000 m²), hard room depth rule, positive gravity, corners equally weighted, 2-dimensional gravity

In [ ]:
paramsB03 = {"experiment": "33.5MA_PG_4CF_2D_3Ca_rdH_block", 
          "iters": 30, 
          "shape": [65, 55, 23, 3, 3, 3], 
          "exceptions": [[10, 0, 0, 5, 100, 2]], 
          "MaximalArea": 16500 / 0.9, 
          "GravityThreshold": 0.2, 
          "GForce": 0.1, 
          "CornerWeights": [1, 1, 1, 1], 
          "corners": True,
          "gravityCenters": 4,
          "dim": 2,
          "CantileverRule": 3,
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}

E03B = gravity_random_generator(paramsB03)

In [ ]:
paramsT03 = {"experiment": "33.5MA_PG_4CF_2D_3Ca_rdH_tower", 
          "iters": 30, 
          "shape": [24, 55, 100 - 23, 3, 3, 3], 
          "exceptions": [], 
          "MaximalArea": 17000 / 0.9, 
          "GravityThreshold": 0.2, 
          "GForce": 0.2, 
          "CornerWeights": [1, 1, 1, 1], 
          "corners": True, 
          "gravityCenters": 4, 
          "dim": 2, 
          "CantileverRule": 2, 
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}
E03T = gravity_random_generator(paramsT03)

### Experiment 4

full area (17500 + 17000 m²), hard room depth rule, positive gravity, corners oriented right, 2-dimensional gravity

In [ ]:
paramsB04 = {"experiment": "34.5MA_PG_4CFR_2D_3Ca_rdH_block", 
          "iters": 30, 
          "shape": [65, 55, 23, 3, 3, 3], 
          "exceptions": [[10, 0, 0, 5, 100, 2]], 
          "MaximalArea": 16500 / 0.9, 
          "GravityThreshold": 0.15,
          "GForce": 0.1, 
          "CornerWeights": [1, 1, -0.2, -0.2], 
          "corners": True,
          "gravityCenters": 4,
          "dim": 2,
          "CantileverRule": 3,
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}

E04B = gravity_random_generator(paramsB04)

In [ ]:
paramsT04 = {"experiment": "34.5MA_PG_4CFR_2D_3Ca_rdH_tower", 
          "iters": 30, 
          "shape": [24, 55, 100 - 23, 3, 3, 3], 
          "exceptions": [], 
          "MaximalArea": 17000 / 0.9, 
          "GravityThreshold": 0.17, 
          "GForce": 0.2, 
          "CornerWeights": [1, 1, -0.2, -0.2], 
          "corners": True, 
          "gravityCenters": 4, 
          "dim": 2, 
          "CantileverRule": 2, 
          "CompactBuildingRule": True, 
          "RoomDepthRule": "hard", 
          "verbose": True}
E04T = gravity_random_generator(paramsT04)